In [1]:
import cv2
import numpy as np
import os

In [2]:
labels=['car', 'truck', 'pedestrian', 'bicyclist', 'light']

In [3]:
width,height=480,300

In [4]:
confThreshold=0.5

In [5]:
nmsThreshold=0.3

In [6]:
modelconf="C:/Users/varun/OneDrive/Desktop/Yolo/DATA/yolov3.cfg"

In [7]:
modelweight="C:/Users/varun/OneDrive/Desktop/Yolo/model/yolov3/backup/yolov3_final.weights"

In [8]:
yolo=cv2.dnn.readNetFromDarknet(modelconf,modelweight)
yolo.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
yolo.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [9]:
def findObjects(outputs,img):
    
    ht,wt,ct=img.shape
    boundingboxes=[]
    classIndexes=[]
    confidences=[]
    
    for output in outputs:
        
        for detection in output:
            
            prob_scores = detection[5:]
            class_index = np.argmax(prob_scores) 
            confidence=prob_scores[class_index]
            
            if confidence>=confThreshold:
                w,h=int(detection[2]*wt),int(detection[3]*ht)
                
                x,y=int((detection[0]*wt)-w/2),int((detection[1]*ht)-h/2)
                
                boundingboxes.append([x,y,w,h])
                classIndexes.append(class_index)
                confidences.append(float(confidence))
                
    indices=cv2.dnn.NMSBoxes(boundingboxes,confidences,confThreshold, nmsThreshold)
    
    for i in indices:
        box=boundingboxes[i]
        x,y,w,h=box[0],box[1],box[2],box[3]
        
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
        cv2.rectangle(img,(x-1,y-25),(x+w+1,y),(255,255,255),-1)
        text=f'{labels[classIndexes[i]].upper()}:{int(confidences[i]*100)}%'
        cv2.putText(img,text,(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.6,(0,0,0),1)
        

In [10]:
img=cv2.imread('C:/Users/varun/OneDrive/Desktop/yolo/images/traffic.jpg')
image=img.copy()

blob=cv2.dnn.blobFromImage(image,1/255,(640,640),(0,0,0),swapRB=True,crop=False)

yolo.setInput(blob)

outputLayersNames=yolo.getUnconnectedOutLayersNames()

print(f"output layers name: {outputLayersNames}")

outputs=yolo.forward(outputLayersNames)

print(f"total number of output: {len(outputs)}")
print(f"shape of first output layer: {outputs[0].shape}")
print(f"shape of second output layer: {outputs[1].shape}")
print(f"shape of third output layer: {outputs[2].shape}")
print(f"inside details of first output layer: {outputs[0][0]}")

findObjects(outputs, image)

cv2.imshow('frame',image)

cv2.waitKey(0)
cv2.destroyAllWindows()

output layers name: ('yolo_82', 'yolo_94', 'yolo_106')
total number of output: 3
shape of first output layer: (1200, 10)
shape of second output layer: (4800, 10)
shape of third output layer: (19200, 10)
inside details of first output layer: [2.4063822e-02 3.1524282e-02 1.4550498e-01 9.3539432e-02 1.1416249e-06
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00]
